# Data Project - Gross unemployment

For this project we use DST API

1. Open the Anaconda prompt.
2. Run the command 'pip install git+https://github.com/alemartinello/dstapi' in the prompt.
3. For a more detailed guide on using the DST API, please refer to https://github.com/alemartinello/dstapi.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dstapi import DstApi

# For interactive plots
import plotly.express as px
import plotly.graph_objs as go

## Defining the problem

In [2]:
# Searching for gross unemployment data through DST API
gross_unemployment_dst = DstApi('AULP01')

# Creating and displaying the a summary of the gross unemployment data
summary_gross_unemployment = gross_unemployment_dst.tablesummary(language='en')
summary_gross_unemployment

Table AULP01: Full-time unemployed persons in per cent of the labour force by region, age, sex and time
Last update: 2023-04-14T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,117,000,All Denmark,997,Abroad,False
1,ALDER,10,TOT,"Age, total",6099,60 year and over,False
2,KØN,3,TOT,Total,K,Women,False
3,Tid,16,2007,2007,2022,2022,True


In [3]:
# Setting parameters
par_gross_unemployment = {
    'table': 'AULP01',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'OMRÅDE', 'values': ['000']}, #000 = All of Denmark
        {'code': 'ALDER', 'values': ['TOT']}, #TOT = Age, total
        {'code': 'KØN', 'values': ['TOT', 'M', 'K']}, #TOT = Total, M = Men, K = Woman
        {'code': 'Tid', 'values': ['2006<=2022']}
    ]
}

# Fetching data based parameters set (Fetching from AUS07)
gross_unemployment = gross_unemployment_dst.get_data(params=par_gross_unemployment)

# Removing the 'OMRÅDE' and 'ALDER' columns
gross_unemployment = gross_unemployment.drop(['OMRÅDE', 'ALDER'], axis=1)

# Sort the DataFrame by date
gross_unemployment = gross_unemployment.sort_values('TID')

# Renaming the columns
gross_unemployment.rename(columns=
                          {'KØN':'GENDER',
                          'TID':'YEAR',
                          'INDHOLD':'GROSS UNEMPLOYMENT RATE (%)'},
                          inplace=True
                    )

# Convert pct. into floats
gross_unemployment['GROSS UNEMPLOYMENT RATE (%)'] = pd.to_numeric(gross_unemployment['GROSS UNEMPLOYMENT RATE (%)'])

# Displaying gross unemployment table
gross_unemployment.head(6)

,GENDER,YEAR,GROSS UNEMPLOYMENT RATE (%)
0,Women,2007,4.4
1,Men,2007,2.9
2,Total,2007,3.6
3,Women,2008,2.9
4,Men,2008,2.4
5,Total,2008,2.6


In [49]:
# Define colors for each line
colors = {'Total': 'green', 'Men': 'blue', 'Women': 'red'}

# Group the data by the 'TOTAL GROSS EMPLOYMENT AND SEX' column
grouped = gross_unemployment.groupby('GENDER')

# Create a figure object
fig_1 = px.line(title='Figure 1: Gross Unemployment as a pct. of the labor force by gender in Denmark')

# Loop through the grouped data and add each group to the figure with the defined color
for name, group in grouped:
    fig_1.add_scatter(x=group['YEAR'], y=group['GROSS UNEMPLOYMENT RATE (%)'], mode='lines', name=name, line=dict(color=colors[name]))

# Update the layout of the figure
fig_1.update_layout(xaxis_title='', yaxis_title='Gross Unemployment Rate (%)', title_font=dict(size=20), 
                  title_x=0.5, margin=dict(l=50, r=50, b=50, t=50, pad=4), hovermode='x unified',
                  xaxis=dict(tickmode='linear', dtick=1),
                  height = 500, width = 1000)

# Add a text box for the source
fig_1.add_annotation(text='Source: Statistics Denmark AULP01', xref='paper', yref='paper', x=0, y=0, showarrow=False, font=dict(size=10))

# Show the interactive plot
fig_1.show()


In [43]:
# Calculating difference between male and female gross unemployment rates
male_unemployment = gross_unemployment[gross_unemployment['GENDER'] == 'Men']
female_unemployment = gross_unemployment[gross_unemployment['GENDER'] == 'Women']

gender_gap = female_unemployment[['YEAR']].copy()
gender_gap['GENDER UNEMPLOYMENT GAP (%)'] = female_unemployment['GROSS UNEMPLOYMENT RATE (%)'].values - male_unemployment['GROSS UNEMPLOYMENT RATE (%)'].values

# Creating the plotly figure object
fig_2 = px.line(gender_gap, x='YEAR', y='GENDER UNEMPLOYMENT GAP (%)', title='Figure 2: Gender Unemployment Gap in Denmark (female unemployment - male unemployment)')

# Updating the layout
fig_2.update_layout(xaxis_title='', yaxis_title='Gender Unemployment Gap (%)', title_font=dict(size=20), 
                  title_x=0.5, margin=dict(l=50, r=50, b=50, t=50, pad=4), hovermode='x unified',
                  xaxis=dict(tickmode='linear', tick0=2007, dtick=1),
                  height = 500, width = 1000)

# Adding horizontal line for y=0
fig_2.add_shape(type='line', x0=2007, y0=0, x1=2022, y1=0, line=dict(color='black', width=0.5))

# Adding text box for source
fig_2.add_annotation(text='Source: Statistics Denmark AULP01', xref='paper', yref='paper', x=0, y=0, showarrow=False, font=dict(size=10))

# Show the interactive plot
fig_2.show()

## Analysis

In [6]:
# Searching for GDP data through DST API
GDP_dst = DstApi('NAN1')

# Creating and displaying the a summary of the GDP data
summary_GDP_dst = GDP_dst.tablesummary(language='en')
summary_GDP_dst

Table NAN1: Demand and supply by transaction, price unit and time
Last update: 2023-03-31T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,TRANSAKT,31,B1GQK,B.1*g Gross domestic product,EMPM_DC,"Total employment (1,000 persons)",False
1,PRISENHED,6,V_M,"Current prices, (bill. DKK.)",LAN_C,"Pr. capita, 2010-prices, chained values, (1000...",False
2,Tid,57,1966,1966,2022,2022,True


In [7]:
# Setting parameters
par_GDP = {
    'table': 'NAN1',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'TRANSAKT', 'values': ['B1GQK']}, # B1GQK
        {'code': 'PRISENHED', 'values': ['L_V']}, #L_V = Period-to-period real growth (pct.)
        {'code': 'Tid', 'values': ['>2006<=2022']}
    ]
}

# Fetching data based parameters set (Fetching from NAN1)
GDP = GDP_dst.get_data(params=par_GDP)

# Removing the 'TRANSAKT' and 'PRISENHED' columns
GDP = GDP.drop(['TRANSAKT', 'PRISENHED'], axis=1)

# Renaming the columns
GDP.rename(columns=
                {'TID':'YEAR',
                'INDHOLD':'REAL GROWTH IN GDP (%)'},
                inplace=True
            )

# Convert pct. into floats
GDP['REAL GROWTH IN GDP (%)'] = pd.to_numeric(GDP['REAL GROWTH IN GDP (%)'])

# Sort the DataFrame by date
GDP = GDP.sort_values('YEAR')

# Displaying GDP tabel
GDP.head(5)

,YEAR,REAL GROWTH IN GDP (%)
0,2007,0.9
1,2008,-0.5
2,2009,-4.9
3,2010,1.9
4,2011,1.3


In [59]:
# Create a bar trace for GDP growth
gdp_trace = go.Bar(x=GDP['YEAR'], y=GDP['REAL GROWTH IN GDP (%)'], name='Real GDP Growth')

# Create a line trace for the gender unemployment gap
gap_trace = go.Scatter(x=gender_gap['YEAR'], y=gender_gap['GENDER UNEMPLOYMENT GAP (%)'], mode='lines', name='Gender Unemployment Gap', line=dict(color='red'))

# Create a list of shapes for the vertical lines
shapes = []
for i in range(1, len(GDP)):
    if GDP['REAL GROWTH IN GDP (%)'][i] > 1.5 and GDP['REAL GROWTH IN GDP (%)'][i-1] <= 1.5:
        shapes.append(dict(type='line', xref='x', yref='paper', x0=GDP['YEAR'][i-1] + 0.5, y0=0, x1=GDP['YEAR'][i-1] + 0.5, y1=1, line=dict(color='magenta')))
    elif GDP['REAL GROWTH IN GDP (%)'][i] <= 1.5 and GDP['REAL GROWTH IN GDP (%)'][i-1] > 1.5:
        shapes.append(dict(type='line', xref='x', yref='paper', x0=GDP['YEAR'][i-1] + 0.5, y0=0, x1=GDP['YEAR'][i-1] + 0.5, y1=1, line=dict(color='magenta')))

# Define the layout
layout = go.Layout(
    title=dict(text='Figure 3: Real GDP Growth and Gender Unemployment Gap in Denmark (2007-2022)', x=0.5, font=dict(size=20)),
    xaxis=dict(title='', range=[2006.5, 2022.5], showticklabels=True),
    yaxis=dict(title='Percentage', range=[-6, 7], tickvals=[-6,-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7]),
    height=700, width=1200,
    shapes=shapes,
    annotations=[
        dict(
            text='Remark: Data for GDP from 2020 to 2022 are preliminary data. Magenta lines indicate change in economy at 1.5%.',
            xref='paper', yref='paper', x=0, y=0.04, showarrow=False, font=dict(size=10)),
        dict(
            text='Source: Statistics Denmark AULP01 & NAN1',
            xref='paper', yref='paper', x=0, y=0.0, showarrow=False, font=dict(size=10))
    ]
)

# Create the figure and add the traces
fig_3 = go.Figure(data=[gdp_trace, gap_trace], layout=layout)

# Show the interactive plot
fig_3.show()

- A positive correlation indicates that the two variables move in the same direction.
- A negative correlation indicates that the two variables move in opposite directions.
- A correlation close to zero indicates there is no strong relationship between the variables.

In [9]:
# Merge the GDP and gender_gap dataframes on 'YEAR'
merged_data = pd.merge(GDP, gender_gap, on='YEAR')

# Calculate the correlation between 'REAL GROWTH IN GDP (%)' and 'GENDER UNEMPLOYMENT GAP (%)'
correlation = merged_data['REAL GROWTH IN GDP (%)'].corr(merged_data['GENDER UNEMPLOYMENT GAP (%)'])

# Print the correlation
print("Correlation between Real GDP Growth and Gender Unemployment Gap:", correlation)

Correlation between Real GDP Growth and Gender Unemployment Gap: 0.3599194870134143


In [10]:
# Searching for GDP data through DST API
employment_dst = DstApi('RAS300')

# Creating and displaying the a summary of the GDP data
summary_employment = employment_dst.tablesummary(language='en')
summary_employment

Table RAS300: Employed (end November) by industry (DB07), socioeconomic status, age, sex and time
Last update: 2022-11-16T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,BRANCHE07,140,TOT,"TOT Industry, total",99999,99999 Activity not stated,False
1,SOCIO,8,05,Self-employed,40,"Employees, not specified",False
2,ALDER,13,-15,Under 16 years,67-,67 years and over,False
3,KØN,3,TOT,Total,K,Women,False
4,Tid,14,2008,2008,2021,2021,True


In [11]:
# Setting parameters
par_employment = {
    'table': 'RAS300',
    'format': 'BULK',
    'lang': 'en',
    'variables': [
        {'code': 'BRANCHE07', 'values': ['1','2','3','4','5','6','7','8','9','10']}, # Choosing all relevant industries
        {'code': 'SOCIO', 'values': ['*']}, # Cannot leave empty so we choose all values with *
        {'code': 'ALDER', 'values': ['*']}, # Cannot leave empty so we choose all values with *
        {'code': 'KØN', 'values': ['TOT', 'M', 'K']}, # TOT = Total, M = Male, K = Female
        {'code': 'Tid', 'values': ['2006<=2021']}
    ]
}

# Fetching data based on parameters set
employment = employment_dst.get_data(params=par_employment)

# Grouping by industry, year, and gender, and summing the NUMBER OF EMPLOYEES column
employment = employment.groupby(['BRANCHE07', 'TID', 'KØN'], as_index=False)['INDHOLD'].sum()

# Remove the numbers at the beginning of the industry names
employment['BRANCHE07'] = employment['BRANCHE07'].str.replace(r'^\d+\s+', '', regex=True)

# Renaming the columns
employment.rename(columns=
                {'BRANCHE07':'INDUSTRY',
                'TID':'YEAR',
                'KØN':'GENDER',
                'INDHOLD':'NUMBER OF EMPLOYEES'},
                inplace=True
            )

# Sorting the DataFrame by industry, year, and gender
employment.sort_values(['INDUSTRY', 'YEAR', 'GENDER'], inplace=True)

# Adding a column with the percentage change in total employment from year to year
employment['CHANGE (%)'] = employment.groupby(['INDUSTRY', 'GENDER'])['NUMBER OF EMPLOYEES'].pct_change() * 100

# Drop rows where YEAR equals 2008
employment = employment[employment['YEAR'] != 2008]

# Displaying employment table
employment.head(6)

,INDUSTRY,YEAR,GENDER,NUMBER OF EMPLOYEES,CHANGE (%)
3,"Agriculture, forestry and fishing",2009,Men,59129,-2.157761
4,"Agriculture, forestry and fishing",2009,Total,74414,-3.077745
5,"Agriculture, forestry and fishing",2009,Women,15285,-6.479442
6,"Agriculture, forestry and fishing",2010,Men,56889,-3.788327
7,"Agriculture, forestry and fishing",2010,Total,71090,-4.466901
8,"Agriculture, forestry and fishing",2010,Women,14201,-7.091920


In [12]:
# Calculate the total number of employees for each industry and year
industry_year_totals = employment.groupby(['INDUSTRY', 'YEAR'])['NUMBER OF EMPLOYEES'].sum().reset_index()

# Pivot the table to have years as columns
industry_year_totals_pivot = industry_year_totals.pivot(index='INDUSTRY', columns='YEAR', values='NUMBER OF EMPLOYEES')

# Calculate the percentage change between the first and last years for each industry
industry_pct_change = (industry_year_totals_pivot.iloc[:, -1] - industry_year_totals_pivot.iloc[:, 0]) / industry_year_totals_pivot.iloc[:, 0] * 100

# Sort the results in descending order
industry_pct_change_sorted = industry_pct_change.sort_values(ascending=False, key=lambda x: abs(x))

# Adding name to the volatility column
industry_pct_change_sorted = industry_pct_change_sorted.reset_index(name='VOLATILITY (%)')

# Create the final list with headers
volatility_list = industry_pct_change_sorted[['INDUSTRY', 'VOLATILITY (%)']].head(10)

# Define the column width
col1_width = max(volatility_list["INDUSTRY"].str.len()) + 2
col2_width = 14

# Creating the table
# Print the header row
print("+" + "-" * col1_width + "+" + "-" * col2_width + "+")
print("|" + "INDUSTRY".ljust(col1_width) + "|" + "VOLATILITY (%)".rjust(col2_width) + "|")
print("+" + "-" * col1_width + "+" + "-" * col2_width + "+")

# Print the table
for idx, row in volatility_list.iterrows():
    industry = row['INDUSTRY']
    volatility = f"{row['VOLATILITY (%)']:.2f}"
    print("|" + f"{industry:<{col1_width}}" + "|" + f"{volatility:>{col2_width}}" + "|")

# Print the footer row
print("+" + "-" * col1_width + "+" + "-" * col2_width + "+")


+-----------------------------------------------------------+--------------+
|INDUSTRY                                                   |VOLATILITY (%)|
+-----------------------------------------------------------+--------------+
|Information and communication                              |         28.60|
|Other business services                                    |         27.61|
|Real estate                                                |         24.04|
|Construction                                               |         18.90|
|Trade and transport etc.                                   |         14.16|
|Arts, entertainment and recration activities               |         12.11|
|Agriculture, forestry and fishing                          |         -9.75|
|Public administration, education and health                |          6.40|
|Manufacturing, mining and quarrying, and utility services  |          2.40|
|Financial and insurance                                    |          0.76|

In [62]:
# Create a pivot table for each industry and year
pivot_table = employment.pivot_table(index='YEAR', columns='INDUSTRY', values='CHANGE (%)')

# Create a line plot for each industry
fig = px.line(pivot_table, x=pivot_table.index, y=pivot_table.columns, title='Figure 3: Percentage Change in Number of Employees by Industry')
fig.update_layout(
    yaxis_title='Percentage Change',
    title_font=dict(size=20),
    title_x=0.5,
    margin=dict(l=50, r=50, b=50, t=50, pad=4),
    height=500, width=1400
)

# Adding text box for source
fig.add_annotation(text='Source: Statistics Denmark AULP01', xref='paper', yref='paper', x=0, y=0, showarrow=False, font=dict(size=10))

# Make the plot interactive
fig.show()